### **Bài 10: Bài tập về nhà: Ứng dụng mô hình sequence2sequence cho bài toán sinh văn bản (text generation)**
Tổng quan: Ở bài tập này chúng ta sẽ ôn lại cách xây dựng và sử dụng mô hình seq2seq cho bài toán sinh văn bản.

**1. Chuẩn bị dữ liệu và tiền xử lý**

Trong bài tập này chúng ta sẽ xử lý dữ liệu của bài toán tóm tắt văn bản để thực nghiệm cho bài toán sinh văn bản. Trong bài toán tóm tắt văn bản, input của chương trình sẽ là 1 văn bản dài và output sẽ là 1 văn bản ngắn hơn và chứa những thông tin quan trọng của văn bản đầu vào. Ngược lại với bài toán trên, trong bài toán sinh văn bản, chúng ta muốn input đầu vào là 1 vài keyword hoặc 1 đoạn văn ngắn và output ra 1 đoạn văn dài. Vì thế, ta hoàn toàn có thể sử dụng dữ liệu trong bài toán tóm tắt văn bản để huấn luyện cho bài toán sinh văn bản, với input là câu đã được tóm tắt và output là đoạn văn gốc.

In [ ]:
import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

Bài tập 1: Bạn hãy download bộ dữ liệu tóm tắt văn bản CNN trong link dưới đây và tiền xử lý chúng.
*   https://huggingface.co/datasets/cnn_dailymail/tree/main/data?fbclid=IwAR2wYgc5NTa0faUrWfPAlUX8yNX_dPztIK58EiS5tTT12NRwYa_dI7QMu_c


Yêu cầu : Sau khi tiền xử lý, chúng ta sẽ có 4 file data gồm :
*   train.input.txt : Chứa các câu tóm tắt dùng để huấn luyện mô hình, thường chiếm 80% kích thước tổng dữ liệu
*   train.output.txt : Chứa các đoạn văn bản gốc ứng với các tóm tắt.
*   valid.input.txt : Chứa các câu tóm tắt dùng để đánh giá mô hình, thường chiếm 10% kích thưởng tổng dữ liệu.
*   valid.output.txt : Chứa các đoạn văn bản gốc ứng vs các tóm tắt.

Lưu ý : Nếu bạn để max_length của dữ liệu quá lớn, thì mô hình của bạn sẽ rất to và có thể gây tràn RAM.






In [ ]:
# YOUR CODE HERE
def create_data(source_path, target_path, number_of_examples):
  # YOUR CODE HERE
  return source_tensors, target_tensors, source_tokenizer, target_tokenizer

number_of_examples = -1
train_src_tensors, train_tgt_tensors, train_src_tokenizer, train_tgt_tokenizer = create_data("train.input.txt", "train.output.txt", number_of_examples)
valid_src_tensors, valid_tgt_tensors, _, _ = create_data("valid.input.txt", "valid.output.txt", -1)

max_length_source, max_length_target = train_src_tensors.shape[1], train_tgt_tensors.shape[1]

print(len(train_src_tensors), len(valid_src_tensors))

**2. Sử dụng tf.data.Dataset để tạo dữ liệu huấn luyện**

Bài tập 2: Hãy xem lại bài tập seq2seq cho bài toán dịch máy để thực hiện cách build data theo batch.

In [ ]:
BUFFER_SIZE = len(train_src_tensors)
BATCH_SIZE = 64

steps_per_epoch = len(train_src_tensors)//BATCH_SIZE
# YOUR CODE HERE


**3. Mô hình Seq2Seq với Attention**

Bài tập 3: Hãy viết lại các thành phần Encoder, Attention, Decoder theo cách hiểu của bạn. Ngoài BahdanauAttention, LuongAttention cũng rất phổ biến, bạn hãy thử implement LuongAttention.

In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, hidden_state_size, batch_sz):
    # YOUR CODE HERE

  def call(self, x, hidden):
    # YOUR CODE HERE

  def initialize_hidden_state(self):
    # YOUR CODE HERE

class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    # YOUR CODE HERE

  def call(self, query, values):
    # YOUR CODE HERE

class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, hidden_state_size, batch_sz):
    # YOUR CODE HERE

  def call(self, x, hidden, enc_output):
    # YOUR CODE HERE

**4. Hàm tối ưu và hàm lỗi**

Hàm tối ưu Adam và hàm lỗi Cross Entropy được dùng rất phổ biến trong các mô hình học sâu, trong phần này, chúng ra sẽ dùng lại các hàm đó nhé.

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  # YOUR CODE HERE

**5. Huấn luyện**

Giờ đã có đủ nguyên liệu và mô hình rồi, ta hãy cùng huấn luyện 1 mô hình có thể sinh văn bản.

In [ ]:
@tf.function
def train_step(source, target, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(source, enc_hidden)
    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([train_tgt_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)

    for t in range(1, target.shape[1]):
      # YOUR CODE HERE

  batch_loss = (loss / int(target.shape[1]))
  variables = encoder.trainable_variables + decoder.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))
  return batch_loss


checkpoint_dir = './model_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

EPOCHS = 1

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (source, target)) in enumerate(train_dataset.take(steps_per_epoch)):
    batch_loss = train_step(source, target, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 1 epochs
  checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

**6. Sinh văn bản**

Hãy sử dụng mô hình vừa được huấn luyện để thực hiện sinh văn bản.

In [ ]:
def evaluate(input_sentence):
  # YOUR CODE HERE
  return result, source_sentence, attention_plot

input_sentence = "hollywood shores up support for ocean 's thirteen"
original_article = "hollywood is planning a new sequel to adventure flick `` ocean 's eleven , '' with star george clooney set to reprise his role as a charismatic thief in `` ocean 's thirteen , '' the entertainment press said wednesday ."
result, input_sentence, attention_plot = evaluate(input_sentence)
print('Input: %s' % (input_sentence))
print('Output : {}'.format(result))